In [1]:
# Step 1: Import the necessary required libraries.
import requests
from requests import get
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

In [23]:
# Step 2: Grab the top 100, sort the result by title.
imdb_url = "https://www.imdb.com/search/title/?groups=top_250&ref_=adv_prv"

In [24]:
# Step 3: Set headers.
headers = {"Accept-Language": "en-US, en;q=0.5"}

In [25]:
# Step 4: Save all values to the results objects coming back from the .get on IMDb URL.
results = requests.get(imdb_url, headers=headers)

In [26]:
# Step 5: Parse the results object to movie_soup using the html parser.
movie_soup = BeautifulSoup(results.text, "html.parser")

In [27]:
# Step 6: I want to extract these attributes (to a list) from the movie_soup.
movie_name = []
movie_years = []
movie_runtime = []
imdb_ratings = []
metascores = []
number_votes = []
us_gross = []


In [28]:
# Step 7: Create a movie_div object to find all div objects in movie_soup.
movie_div = movie_soup.find_all('div', class_='lister-item mode-advanced')

In [29]:
# Step 8: Loop through each object in the movie_div.
for container in movie_div:

# Step 9: Add each result from each attribute for each list.

        # name
        name = container.h3.a.text
        movie_name.append(name)

        # year
        year = container.h3.find('span', class_='lister-item-year').text
        movie_years.append(year)

        # runtime
        runtime = container.p.find('span', class_='runtime').text if container.p.find('span', class_='runtime').text else '-'
        movie_runtime.append(runtime)

        # IMDB rating
        imdb = float(container.strong.text)
        imdb_ratings.append(imdb)

        # metascore
        m_score = container.find('span', class_='metascore').text if container.find('span', class_='metascore') else '-'
        metascores.append(m_score)

        # There are two NV containers, grab both of them as they hold both the votes and the grosses
        nv = container.find_all('span', attrs={'name': 'nv'})

        # filter nv for votes
        vote = nv[0].text
        number_votes.append(vote)

        # filter nv for gross
        grosses = nv[1].text if len(nv) > 1 else '-'
        us_gross.append(grosses)

In [30]:
# Step 10: Build and store all of the attributes into the Pandas movie dataframe.        
movies = pd.DataFrame({
'movie_name': movie_name,
'movie_year': movie_years,
'movie_runtime': movie_runtime,
'imdb_ratings': imdb_ratings,
'metascore': metascores,
'number_votes': number_votes,
'us_gross_millions': us_gross,
})

In [31]:
movies.head()

,movie_name,movie_year,movie_runtime,imdb_ratings,metascore,number_votes,us_gross_millions
0,Top Gun: Maverick,(2022),130 min,8.3,78,"549,703",$718.73M
1,The Godfather,(1972),175 min,9.2,100,"1,886,778",$134.97M
2,Rocky,(1976),120 min,8.1,70,"590,143",$117.24M
3,The Shawshank Redemption,(1994),142 min,9.3,82,"2,715,781",$28.34M
4,Interstellar,(2014),169 min,8.6,74,"1,875,492",$188.02M


In [32]:
# Step 11: Use Pandas str.extract to remove all String characters, and save the value as type int for cleaning up the data with Pandas.
#movies['movie_year'] = movies['movie_year'].str.extract('(\d+)').astype(int)
movies['movie_year'] = movies['movie_year'].astype(int)
movies['movie_runtime'] = movies['movie_runtime'].astype(int)


ValueError: invalid literal for int() with base 10: '(2022)'

In [22]:
movies[ movies['movie_year'] >= 2018 ]

,movie_name,movie_year,movie_runtime,imdb_ratings,metascore,number_votes,us_gross_millions
9,Parasite,2019,132,8.5,96,"832,310",$53.37M
10,Avengers: Endgame,2019,181,8.4,78,"1,158,610",$858.37M
13,Joker,2019,122,8.4,59,"1,325,069",$335.45M
40,Avengers: Infinity War,2018,149,8.4,68,"1,104,091",$678.82M
44,Spider-Man: Into the Spider-Verse,2018,117,8.4,87,"547,571",$190.24M


In [18]:
movies['metascore'] = movies['metascore'].astype(int)
movies['number_votes'] = movies['number_votes'].astype(int)

movies['us_gross_millions'] = movies['us_gross_millions'].map(lambda x: x.lstrip('$').rstrip('M'))
movies['us_gross_millions'] = pd.to_numeric(movies['us_gross_millions'], errors='coerce')

ValueError: invalid literal for int() with base 10: '-'

In [ ]:
# Step 12: Export our movie results to a pretty little .csv file.
movies.to_csv('top_100_movies.csv')